In [7]:
install.packages("shinylive")
install.packages("shiny")
install.packages("shinyWidgets")
install.packages("dplyr")
install.packages("ggplot2")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘colorspace’, ‘farver’, ‘labeling’, ‘munsell’, ‘RColorBrewer’, ‘viridisLite’, ‘gtable’, ‘isoband’, ‘scales’




---
title: "Template for r-shinylive Quarto document (Covid Example)"
format:
  html:
    resources: 
      - shinylive-sw.js
filters:
  - shinylive
---

In [10]:
options(timeout = 6000)
library(shiny)
library(dplyr)
library(ggplot2)
library(shinyWidgets)

# Download real COVID data for Washington state counties from the New York Times GitHub repo
covid_url <- "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
covid_data <- read.csv(covid_url)

In [11]:

# Filter for Washington state and prepare the data
washington_covid_data <- covid_data %>%
  filter(state == "Washington") %>%
  select(date, county, cases) %>%
  mutate(year_month = format(as.Date(date), "%Y%m"))  # Create year_month column

# Create a vector of unique year_month values from the dataset
valid_year_months <- sort(unique(washington_covid_data$year_month))
valid_year_months_index <- seq_along(valid_year_months)

# Define the UI
ui <- fluidPage(
  titlePanel("COVID Case Counts by Month-Year in Washington Counties"),
  
  # Use fluidRow to place the slider at the top, spanning the entire page width
  fluidRow(
    column(12,  # 12 columns to span full width
      sliderTextInput("selected_year_month", 
                      "Select Year and Month:", 
                      choices = valid_year_months,  # Use year-month values directly
                      selected = "202001",  # Start with the first year-month
                      animate = animationOptions(interval = 1000, loop = FALSE))
    )
  ),
  
  # Main panel for the plot below the slider
  fluidRow(
    column(12,  # This will also span the entire width
      plotOutput("histogramPlot")
    )
  )
)


# Define the server
server <- function(input, output) {
  output$histogramPlot <- renderPlot({
    filtered_data <- washington_covid_data %>%
      filter(year_month == as.character(input$selected_year_month)) %>%
      group_by(county) %>%
      summarize(total_cases = max(cases, na.rm = TRUE))  # Get max case count for each county in the selected year_month
    
    ggplot(filtered_data, aes(x = county, y = total_cases)) +
      geom_bar(stat = "identity", fill = "steelblue") +
      labs(title = paste("COVID Cases in", input$selected_year_month),
           x = "County", 
           y = "Total Case Count") +
      theme_minimal() +
      theme(axis.text.x = element_text(angle = 45, hjust = 1))
  })
}

# Run the Shiny app
shinyApp(ui = ui, server = server)


Listening on http://127.0.0.1:7477



In [12]:
getwd()

[1] "/workspaces/quarto_demo/.devcontainer"